In [1]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
image_width, image_height = 64, 64
num_classes = 165
images = []
labels = []
class_label_mapping = {}
class_count = 0

In [3]:
dataset_path = r'D:\Project_3\Deep_Bais_FPT_PJ3\tai\EarVN1.0 dataset\Images'

In [4]:
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((image_width, image_height))
    img = np.array(img)
    img = img / 255.0 
    return img

In [5]:
for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_folder):
        continue

    class_label_mapping[class_name] = class_count
    class_count += 1

    for image_name in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_name)
        img = preprocess_image(image_path)
        images.append(img)
        labels.append(class_label_mapping[class_name])

In [6]:
images = np.array(images)
labels = np.array(labels)

In [7]:
# One-hot encode the labels
labels = to_categorical(labels, num_classes)

In [8]:
train_images, temp_images, train_labels, temp_labels = train_test_split(images, labels, test_size=0.3, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=0.5, random_state=42)

In [9]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_datagen.fit(train_images)

In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

In [13]:
model.fit(train_datagen.flow(train_images, train_labels, batch_size=32), epochs=100, validation_data=(val_images, val_labels))

Epoch 1/100
622/622 [==============================] - 23s 25ms/step - loss: 5.0797 - accuracy: 0.0110 - val_loss: 5.0257 - val_accuracy: 0.0127
Epoch 2/100
622/622 [==============================] - 15s 25ms/step - loss: 5.0087 - accuracy: 0.0144 - val_loss: 4.9553 - val_accuracy: 0.0134
Epoch 3/100
622/622 [==============================] - 15s 24ms/step - loss: 4.9865 - accuracy: 0.0141 - val_loss: 4.9391 - val_accuracy: 0.0183
Epoch 4/100
622/622 [==============================] - 15s 24ms/step - loss: 4.9587 - accuracy: 0.0160 - val_loss: 4.8791 - val_accuracy: 0.0214
Epoch 5/100
622/622 [==============================] - 14s 23ms/step - loss: 4.9246 - accuracy: 0.0186 - val_loss: 4.8040 - val_accuracy: 0.0275
Epoch 6/100
622/622 [==============================] - 15s 23ms/step - loss: 4.8804 - accuracy: 0.0202 - val_loss: 4.7668 - val_accuracy: 0.0324
Epoch 7/100
622/622 [==============================] - 16s 25ms/step - loss: 4.8602 - accuracy: 0.0218 - val_loss: 4.7526 - val_ac

In [14]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

134/134 [==============================] - 0s 3ms/step - loss: 3.9580 - accuracy: 0.1405
Test Loss: 3.9579625129699707, Test Accuracy: 0.1405443400144577
